# Atomate2 introduction

Atomate2 is an open-source library providing computational workflows for automating first-principles calculations.

## Supported DFT Codes

- ABINIT
- CP2K
- FHI-AIMS
- JDFTx
- Q-Chem
- VASP

  
## Workflows

Some of the workflows available in atomate2 are:

- electronic band structures
- elastic, dielectric, and piezoelectric tensors
- one-shot electron-phonon interactions
- electronic transport using [AMSET](https://hackingmaterials.lbl.gov/amset/)
- phonons and Grüneisen parameter computations using [phonopy](https://github.com/phonopy/phonopy)
- defect formation energy diagrams
- [Lobster](http://www.cohp.de/) bonding analysis with [LobsterPy](https://github.com/JaGeo/LobsterPy)

## MLIP based workflows

Workflows using MLIPs like MACE, CHGNET, M3GNET, GAP, NEP, SevenNet are available for:

- phonons and Grüneisen parameter computations
- elastic tensors


It is easy to customise and compose any of the above workflows.

# Installation and Setup 

Typically, before starting to use workflows available in atomate2, you will need to create an python environment with necessary packages and also set some configuration files where you specify details like path to VASP binary, your database information like collection names, username, password etc. You can find this in the [atomate2 documentation](https://materialsproject.github.io/atomate2/user/install.html). If interested in specifics , let us know we are happy to provide more information.


For this school this has been setup already for you, so we can directly create workflows and submit it to the hpc-cluster. One can also use [Fireworks](https://materialsproject.github.io/atomate2/user/fireworks.html) to manage and execute your workflows. In this school we will use [jobflow-remote](https://github.com/Matgenix/jobflow-remote) for managing and submitting our workflows which has been introduced in earlier sessions. 

# Running a Phonon workflow with VASP and accessing the results

In [ ]:
from pymatviz.structure_viz import structure_3d_plotly
from atomate2.vasp.flows.phonons import PhononMaker
from atomate2.vasp.powerups import add_metadata_to_flow
from pymatgen.core import Structure
from jobflow_remote import submit_flow

In [ ]:
# Read si structure
si_structure = Structure.from_file("Si.vasp")

In [ ]:
structure_3d_plotly(si_structure)

In [ ]:
# make a Phonon flow to optimise the structure and obtain the Phonon bandstructure
# It is possible to further customize calculation parameters, but for this case we stick to defaults
phonon_flow = PhononMaker(
    min_length=3.0, # here we set small supercell length for faster calculations (ideally you would use 15-20 angstroms supercell)
    use_symmetrized_structure="conventional",
    create_thermal_displacements=False,
    store_force_constants=False,
    prefer_90_degrees=False,
    generate_frequencies_eigenvectors_kwargs={"tstep": 100, "band_structure_eigenvectors": False},
).make(si_structure)

In [ ]:
resources={"nodes": 1 , "ntasks": 36, "time": "03:00:00"} # define the resources for the jobs in flow

In [ ]:
submit_flow(phonon_flow, worker="cecam", resources=resources, exec_config="vasp_6.4.3_cecam") # set the resources for job execution

```bash
jf runner start # this is optional (if the jobflow remote runner is running not needed to execute)
```

In [ ]:
# get overview of the list of jobs added and its status
! jf job list

If you prefer to monitor jobs via a graphical user interface simply run. This should start a gui server which can be opened in a browser

```bash
jf gui
```

## Accessing the phonon workflow results from the database and plotting

Each of the atomate2 workflows mostly have a associated [Taskdocument](https://materialsproject.github.io/atomate2/user/key_concepts_overview.html) generated for specific type of calculations. The Taskdocument can be understood simply as summary of most important results from the calculation.

For phonon workflow, the taskdocument is named as [PhononBSDOSDoc](https://github.com/materialsproject/atomate2/blob/b8ff9f444bcecaac474f3779acd83ee3b5ad4ae1/src/atomate2/common/schemas/phonons.py#L136). 

So now, we will access this document which is stored in the database after successful run of the workflow.

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
from pymatgen.phonon.bandstructure import PhononBandStructureSymmLine
from pymatgen.phonon.dos import PhononDos
from pymatgen.phonon.plotter import PhononBSPlotter, PhononDosPlotter

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
# get output doc

result = jobstore.query(criteria={"metadata.db_id": "17"}, load=True)

for res in result:
    output_doc = MontyDecoder().process_decoded(res)

In [ ]:
# initialize dos plotter and visualize dos plot
dos_plot = PhononDosPlotter()
dos_plot.add_dos(label="Total Dos", dos=output_doc.output.phonon_dos)
dos_plot.get_plot();

# initialize Phonon bandstructure plotter and visualize band structure plot
bs_plot = PhononBSPlotter(bs=output_doc.output.phonon_bandstructure)
bs_plot.get_plot();

# Running a Lobster workflow with VASP and accessing the results

In [ ]:
from pymatviz.structure_viz import structure_3d_plotly
from atomate2.vasp.flows.mp import MPVaspLobsterMaker
from atomate2.vasp.powerups import add_metadata_to_flow, update_user_incar_settings
from pymatgen.core import Structure
from jobflow_remote import submit_flow

In [ ]:
# Read AlAgS2 structure
AlAgS2_structure = Structure.from_file("AlAgS2.vasp").get_primitive_structure()

In [ ]:
structure_3d_plotly(AlAgS2_structure)

In [ ]:
lobster_flow = MPVaspLobsterMaker(relax_maker=None).make(structure=AlAgS2_structure)

lobster_flow = update_user_incar_settings(lobster_flow, {"NPAR": 6, "ALGO": "FAST"})

In [ ]:
resources={"nodes": 1 , "ntasks": 36, "time": "03:00:00"} # define the resources for the jobs in flow

In [ ]:
submit_flow(lobster_flow, worker="cecam", resources=resources, exec_config="vasp_6.4.3_cecam") # set the resources for job execution

In [ ]:
# get overview of the list of jobs added and its status
! jf job list

## Accessing the lobster workflow results from the database and plotting

For Lobster workflow, the taskdocument is named as [LobsterTaskDocument](https://github.com/materialsproject/atomate2/blob/b8ff9f444bcecaac474f3779acd83ee3b5ad4ae1/src/atomate2/lobster/schemas.py#L640).

So now, we will access this document which is stored in the database after successful run of the workflow.

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
from pymatgen.electronic_structure.cohp import Cohp
from lobsterpy.plotting import PlainCohpPlotter, InteractiveCohpPlotter

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
# get output doc
result = jobstore.query(criteria={"metadata.db_id": "6"}, load=True)

for res in result:
    output_doc = MontyDecoder().process_decoded(res)

In [ ]:
for key, cohp in output_doc.output.lobsterpy_data_cation_anion.cohp_plot_data.data.items():
    plotter = PlainCohpPlotter()
    plotter.add_cohp(key, cohp)
    plotter.get_plot(sigma=0.05, ylim=[-10,5])

In [ ]:
# access text description
output_doc.output.lobsterpy_text_cation_anion

In [ ]:
# access calculation quality summary
output_doc.output.calc_quality_text